# NeRF Training with Nerfstudio

**Simple NeRF training for comparison with 3DGS**

**Expected Time:** ~30 minutes for 10k iterations

## Step 1: Check GPU

In [ ]:
!nvidia-smi

import torch
print(f"\nCUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Step 2: Install Nerfstudio

In [ ]:
!pip install nerfstudio -q
print("✓ Nerfstudio installed")

## Step 3: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("✓ Drive mounted")

## Step 4: Copy Dataset from Drive

In [ ]:
import os

# Create data directory
!mkdir -p data/nerf_synthetic

# Copy lego dataset
!cp -r /content/drive/MyDrive/data/nerf_synthetic/lego data/nerf_synthetic/

print("✓ Dataset copied")
!ls data/nerf_synthetic/lego/

## Step 5: Fix Dataset JSON Files

Your dataset has different filenames than expected, so we need to fix the JSON files

In [ ]:
import json
import os

# Fix JSON files to match actual image filenames
for split in ['train', 'val', 'test']:
    json_path = f'data/nerf_synthetic/lego/transforms_{split}.json'
    
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Get actual image files (excluding depth maps)
    split_dir = f'data/nerf_synthetic/lego/{split}'
    actual_files = sorted([f.replace('.png', '') for f in os.listdir(split_dir) 
                          if f.endswith('.png') and 'depth' not in f])
    
    print(f"{split}: {len(actual_files)} images (first: {actual_files[0] if actual_files else 'none'})")
    
    # Update frame paths
    for i, frame in enumerate(data['frames']):
        if i < len(actual_files):
            data['frames'][i]['file_path'] = f"./{split}/{actual_files[i]}"
    
    # Save fixed JSON
    with open(json_path, 'w') as f:
        json.dump(data, f, indent=2)
    
    print(f"  ✓ Fixed {split}")

print("\n✓ All dataset files ready!")

## Step 6: Patch Nerfstudio for PyTorch 2.6 Compatibility

Fix checkpoint loading issue with newer PyTorch

In [ ]:
# Patch eval_utils.py to fix weights_only error
eval_utils_path = '/usr/local/lib/python3.12/dist-packages/nerfstudio/utils/eval_utils.py'

with open(eval_utils_path, 'r') as f:
    content = f.read()

# Fix the torch.load line
if 'weights_only=False' not in content:
    content = content.replace(
        'loaded_state = torch.load(load_path, map_location="cpu")',
        'loaded_state = torch.load(load_path, map_location="cpu", weights_only=False)'
    )
    
    with open(eval_utils_path, 'w') as f:
        f.write(content)
    
    print("✓ Patched Nerfstudio for PyTorch 2.6")
else:
    print("✓ Already patched")

## Step 7: Train NeRF

**This will take ~30 minutes for 10,000 iterations**

Keep the browser tab open!

In [ ]:
# Train NeRF with nerfacto
!ns-train nerfacto \
    --data data/nerf_synthetic/lego \
    --pipeline.model.camera-optimizer.mode off \
    --max-num-iterations 10000 \
    --viewer.quit-on-train-completion True \
    blender-data

## Step 8: Find the Trained Model

In [ ]:
import glob

# Find the config file
configs = glob.glob('outputs/lego/nerfacto/*/config.yml')
if configs:
    nerf_config = configs[0]
    print(f"✓ Found model: {nerf_config}")
else:
    print("✗ Model not found!")
    !find outputs -name "config.yml" 2>/dev/null

## Step 9: Evaluate NeRF on Test Set

Get PSNR, SSIM, and LPIPS metrics

In [ ]:
# Evaluate on test set
!ns-eval \
    --load-config {nerf_config} \
    --output-path nerf_results.json

print("\n✓ Evaluation complete!")

## Step 10: View Results

In [ ]:
import json

# Load results
with open('nerf_results.json', 'r') as f:
    results = json.load(f)

print("="*60)
print("NERF RESULTS (10,000 iterations)")
print("="*60)
print(f"PSNR:  {results['results']['psnr']:.2f} dB")
print(f"SSIM:  {results['results']['ssim']:.4f}")
print(f"LPIPS: {results['results']['lpips']:.4f}")
print(f"\nNumber of test images: {results['results']['num_rays']}")
print("="*60)

# Save formatted results
with open('nerf_metrics_summary.txt', 'w') as f:
    f.write("NeRF Training Results\n")
    f.write("=" * 60 + "\n")
    f.write(f"Method: Nerfacto (Nerfstudio)\n")
    f.write(f"Iterations: 10,000\n")
    f.write(f"Dataset: NeRF Synthetic LEGO\n\n")
    f.write(f"PSNR:  {results['results']['psnr']:.2f} dB\n")
    f.write(f"SSIM:  {results['results']['ssim']:.4f}\n")
    f.write(f"LPIPS: {results['results']['lpips']:.4f}\n")

print("\n✓ Summary saved to nerf_metrics_summary.txt")

## Step 11: Render Test Images

In [ ]:
# Render test set images
!mkdir -p renders

!ns-render camera-path \
    --load-config {nerf_config} \
    --camera-path-filename data/nerf_synthetic/lego/transforms_test.json \
    --output-path renders/

print("\n✓ Test images rendered to renders/")

## Step 12: View Sample Renders

In [ ]:
from IPython.display import Image, display
import glob

# Get rendered images
rendered_imgs = sorted(glob.glob('renders/*.png'))[:5]

print(f"Sample renderings ({len(rendered_imgs)} shown):\n")
for img_path in rendered_imgs:
    print(f"\n{img_path}:")
    display(Image(img_path, width=400))

## Step 13: Download Everything

In [ ]:
from google.colab import files

# Zip all results
!zip -r nerf_complete_results.zip \
    nerf_results.json \
    nerf_metrics_summary.txt \
    renders/ \
    outputs/lego/nerfacto/

print("\n✓ Results zipped")
print("\nContents:")
!zipinfo nerf_complete_results.zip | head -30

# Download
print("\nStarting download...")
files.download('nerf_complete_results.zip')

print("\n✓ Download complete!")

## Step 14: Summary for Report

In [ ]:
# Print final summary
print("="*60)
print("FINAL SUMMARY FOR YOUR REPORT")
print("="*60)
print(f"\nMethod: NeRF (Nerfacto implementation via Nerfstudio)")
print(f"Training: 10,000 iterations (~30 minutes on T4 GPU)")
print(f"Dataset: NeRF Synthetic LEGO (Blender)")
print(f"\nTest Set Results:")
print(f"  PSNR:  {results['results']['psnr']:.2f} dB")
print(f"  SSIM:  {results['results']['ssim']:.4f}")
print(f"  LPIPS: {results['results']['lpips']:.4f} (lower is better)")
print(f"\nCompare these metrics with your teammate's 3DGS results!")
print("\n" + "="*60)
print("\n✓ All done! Download the zip file above.")
print("="*60)